In [ ]:
## All required imports for the notebook
from google.colab import files
import zipfile
import xml.etree.ElementTree as ET

import PIL
from PIL import Image

import os
from google.colab import drive
drive.mount('/content/gdrive')

import json

Mounted at /content/gdrive


In [ ]:
## file uploader from disk - for XML zip file
uploaded = files.upload()

Saving xmldata.zip to xmldata.zip


In [ ]:
## create dictionaries for Image BBox coordinates and other data
imgBoxDict = {}
imgOtherParametersDict = {}

In [ ]:
## created constructor for this type object "ImgOtherParameters"
## initialising the objects that the value is being passed to - using self property folder to link to the new one made with the creation 
class ImgOtherParameters:
  def __init__(self, folder, filename, path, database, depth, segmented, name, pose, truncated, difficult): 
    self.folder = folder
    self.filename = filename
    self.path = path
    self.database = database
    self.depth = depth
    self.segmented = segmented
    self.name = name
    self.pose = pose
    self.truncated = truncated
    self.difficult = difficult

In [ ]:
##create class of Box coordinates, to be changed later that can be joined with the other data and compiled after the resize
class Box:
  def __init__(self, width, height, xmin, ymin, xmax, ymax):
    self.width = int(width)
    self.height = int(height)
    self.xmin = int(xmin)
    self.ymin = int(ymin)
    self.xmax = int(xmax)
    self.ymax = int(ymax)

## created the override of __str__ method, to redefine the way on how the Box will be print
  def __str__(self): 
    return str(self.width) + "; "+ str(self.height) + "; " + str(self.xmin) + "; " + str(self.ymin) + "; " + str(self.xmax) + "; " + str(self.ymax) + ";"

In [ ]:
## read the zip file of the XML data
zf = zipfile.ZipFile('xmldata.zip', 'r')

#Parse the XML files with a step of 2 by 2 to skip mac created files
for name in [zf.namelist()[i] for i in range(len(zf.namelist())) if i % 2 == 0][2:]:
  if '0' in name:
    tree = ET.parse(zf.open(name))
    root = tree.getroot()

    ## find the root of each part of the Element Tree of the XML files and assign to variable 
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    xmin = root.find('object').find('bndbox').find('xmin').text
    ymin = root.find('object').find('bndbox').find('ymin').text
    xmax = root.find('object').find('bndbox').find('xmax').text
    ymax = root.find('object').find('bndbox').find('ymax').text

    ## Assigns assigned variables to the file name and adds to dictionary
    imgBoxDict[root.find('filename').text] = Box(width, height, xmin, ymin, xmax, ymax)

    folder = root.find('folder').text
    filename = root.find('filename').text
    path = root.find('path').text
    database = root.find('source').find('database').text
    depth = root.find('size').find('depth').text
    segmented = root.find('segmented').text
    name = root.find('object').find('name').text
    pose = root.find('object').find('pose').text
    truncated = root.find('object').find('truncated').text
    difficult = root.find('object').find('difficult').text

    imgOtherParametersDict[root.find('filename').text] = ImgOtherParameters(folder, filename, path, database, depth, segmented, name, pose, truncated, difficult)

In [ ]:
## print each item in the images dictionary along with the BBox variables assigned above 
for i in imgBoxDict:
  print(i + ": " + str(imgBoxDict[i]))

01.jpg: 1100; 618; 269; 39; 826; 600;
03.jpg: 637; 421; 439; 81; 482; 127;
02.jpg: 2000; 1333; 237; 319; 1018; 1172;
06.jpg: 640; 427; 43; 226; 164; 339;
05.jpg: 1200; 817; 919; 622; 999; 703;
04.jpg: 1920; 1200; 1041; 698; 1753; 1079;


In [ ]:
# create variables for final image sizes - used to resize
x_final = 800
y_final = 450

In [ ]:
# Generate dictionary for final images after resizing 
imgBoxDictFinal = {}

In [ ]:
# Scale_factor used to calculate the new bbox coordinates of the resized images
for i in imgBoxDict:
  x_scale_factor = x_final / imgBoxDict[i].width 
  y_scale_factor = y_final / imgBoxDict[i].height  

  # calculate the new coordinates
  xmin_new = imgBoxDict[i].xmin * x_scale_factor
  xmax_new = imgBoxDict[i].xmax * x_scale_factor
  ymin_new = imgBoxDict[i].ymin * y_scale_factor
  ymax_new = imgBoxDict[i].ymax * y_scale_factor

  # adds new coordinates to the final dictionary
  imgBoxDictFinal[i] = Box(x_final, y_final, xmin_new, ymin_new, xmax_new, ymax_new)

In [ ]:
# prints old box coordinates in first for loop
# seccond for loop prints the new coordinates 
for i in imgBoxDict:
  print(i + ": " + str(imgBoxDict[i]))

for i in imgBoxDictFinal:
  print(i + ": " + str(imgBoxDictFinal[i]))

01.jpg: 1100; 618; 269; 39; 826; 600;
03.jpg: 637; 421; 439; 81; 482; 127;
02.jpg: 2000; 1333; 237; 319; 1018; 1172;
06.jpg: 640; 427; 43; 226; 164; 339;
05.jpg: 1200; 817; 919; 622; 999; 703;
04.jpg: 1920; 1200; 1041; 698; 1753; 1079;
01.jpg: 800; 450; 195; 28; 600; 436;
03.jpg: 800; 450; 551; 86; 605; 135;
02.jpg: 800; 450; 94; 107; 407; 395;
06.jpg: 800; 450; 53; 238; 205; 357;
05.jpg: 800; 450; 612; 342; 666; 387;
04.jpg: 800; 450; 433; 261; 730; 404;


In [ ]:
# read the image folder stored in the GDrive
f = r'/content/gdrive/MyDrive/Internship/images'

In [ ]:
# print the list directory of the image names within the file 
os.listdir(f)

['05.jpg', '06.jpg', '04.jpg', '02.jpg', '01.jpg', '03.jpg']

In [ ]:
# attempt to create new folder for resized images 
# open each file, reset the size to the defined sizes
# save resized files in the new folder 
try:
  os.mkdir("/content/gdrive/MyDrive/Internship/imagesresized")
except:
  print("Folder already exists")

for file in os.listdir(f):
  img = Image.open(f+"/"+file)
  img = img.resize((x_final, y_final))
  img.save(f+"resized/"+file + ' resized.jpg', 'JPEG', quality=90)

Folder already exists


In [ ]:
# concatenates all image data from dictionary 

images = ""
for i in imgBoxDictFinal:
  images += ('{"folder":"'+imgOtherParametersDict[i].folder+
             '", "filename":"'+imgOtherParametersDict[i].filename+
             '", "path":"'+imgOtherParametersDict[i].path+
             '", "source": { "database": "'+imgOtherParametersDict[i].database+
             '"}, "size": { "width": "'+str(imgBoxDictFinal[i].width)+'", "height": "'+str(imgBoxDictFinal[i].height)+'", "depth": "'+imgOtherParametersDict[i].depth+
             '"}, "segmented":"'+imgOtherParametersDict[i].segmented+
             '", "object": { "name": "'+imgOtherParametersDict[i].name+'", "pose": "'+imgOtherParametersDict[i].pose+'", "truncated": "'+imgOtherParametersDict[i].truncated+'", "difficult": "'+imgOtherParametersDict[i].difficult+'", "bndbox": { "xmin": "'+str(imgBoxDictFinal[i].xmin)+'", "ymin": "'+str(imgBoxDictFinal[i].ymin)+'", "xmax": "'+str(imgBoxDictFinal[i].xmax)+'", "ymax": "'+str(imgBoxDictFinal[i].ymax)+
             '"}}},')

# conversion of json ad hoc string into an effective JSON file
jsonFile = json.dumps(json.loads("{\"images\": ["+images[:-1]+"]}"))

#writes and encodes the JSON file and saves as a .txt file
with open('json.txt', 'w') as f:
  f.write(str(jsonFile.encode('utf-8')))

# downloads the .txt file
files.download('json.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>